# Ball in Bag Problem

Pada sekitar tahun 1700, Jacob Bernoulli menulis sebuah kasus mengenai pengambilan bola berwarna dari sebuah kantong, dan sejak saat itu, penjelasan mengenai probabilitas sering memakai contoh ini.

<!-- <img src="https://blue.kumparan.com/image/upload/fl_progressive,fl_lossy,c_fill,q_auto:best,w_640/v1533719610/Jakob_Bernoulli_vufttj.jpg" width="300"/>
<center><a href="https://en.wikipedia.org/wiki/Jacob_Bernoulli">Jacob Bernoulli</a><br>1700</center> -->

Sebagai contoh, berikut adalah permasalahan yang diadaptasi dari [mathforum.org](http://mathforum.org/library/drmath/view/69151.html):

> *Dalam sebuah kantong terdapat bola berwarna dengan jumlah 6 biru, 9 merah, dan 8 putih. Dari kantong tersebut diambil 6 bola random. Berapa probabilitas dari masing-masing hasil dibawah ini:*

> - *Semua bola merah*.
> - *3 biru, 1 merah, dan 2 putih*.
> - *4 bola putih*.

Untuk simulasi, kita akan definisikan isi dari kantong. Sebuah `set` tidak dapat memiliki elemen yang sama persis. Sehingga penamaan harus berbeda, misal bola biru diberi kode `'B1'` sampai `'B6'`, bukan 6 elemen yang sama dengan nama `'B'`:

## Previous Defined Functions

In [1]:
import itertools
import random
from fractions import Fraction

In [2]:
def P(event, space): 
    "The probability of an event, given a sample space."
    favorable = set.intersection # Outcomes that are in the event and in the sample space
    cases     = len              # The number of cases is the length, or size, of a set
    return Fraction(cases(favorable(event, space)), 
                    cases(space))

def combos(items, n):
    "All combinations of n items; each combo as a space-separated str."
    return set(map(' '.join, itertools.combinations(items, n)))


## Balls in Bag

In [3]:
def balls(color, n):
    "A set of n numbered balls of the given color."
    return {color + str(i)
            for i in range(1, n + 1)}

urn = balls('B', 6) | balls('R', 9) | balls('W', 8)
urn

{'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'R1',
 'R2',
 'R3',
 'R4',
 'R5',
 'R6',
 'R7',
 'R8',
 'R9',
 'W1',
 'W2',
 'W3',
 'W4',
 'W5',
 'W6',
 'W7',
 'W8'}

Sekarang kita definisikan ruang sampel, `U6`, sebagai set dari kombinasi 6 bola:  

In [4]:
U6 = combos(urn, 6)

random.sample(U6, 5)

C:\Users\Acer\AppData\Local\Temp\ipykernel_10740\409875261.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(U6, 5)


['W3 R4 W1 W6 W5 R1',
 'R8 R4 B4 W4 R2 R3',
 'R5 W1 R9 R7 R3 B1',
 'R5 W3 W1 W6 W4 B5',
 'R8 B4 R9 R7 W5 B5']

Definisikan fungsi  `select` sehingga `select('R', 6)` adalah kejadian mengambil 6 bola merah dari kantong:

In [5]:
def select(color, n, space=U6):
    "The subset of the sample space with exactly `n` balls of given `color`."
    return {s for s in space if s.count(color) == n}

Sekarang, kita bisa menjawab 3 pertanyaan tadi:

> *Semua bola merah*.

In [6]:
P(select('R', 6), U6) 

Fraction(4, 4807)

> *3 biru, 1 merah, dan 2 putih*.

In [7]:
P(select('B', 3)  & select('R', 1) & select('W', 2), U6)

Fraction(240, 4807)

> *4 bola putih*.

In [8]:
P(select('W', 4), U6)

Fraction(350, 4807)

## Balls in Bag via arithmetic

Kita dapat memverifikasi perhitungan diatas dengan menggunakan aritmatik. Pertama, berapa cara kita bisa memilih 6 dari 9 bola merah? Untuk bola pertama, kita dapat memilih bola merah mana saja dari 9 yang ada, lalu pilih satu lagi dari 8 sisanya, dan seterusnya sampai terpilih 6 bola. Namun, dalam kasus ini urutan dari bola tidak diperhatikan, Sehingga dalam kasus ini dapat digunakan **kombinasi**. Secara umum, jumlah cara untuk mengambil *c* dari *n* item adalah: 

(*n* choose *c*) = *n*! / ((*n* - *c*)! &times; c!).

Kita dapat menerjemahkan menjadi sebuah kode:

In [9]:
from math import factorial

def choose(n, c):
    "Number of ways to choose c items from a list of n items."
    return factorial(n) // (factorial(n - c) * factorial(c))

In [10]:
choose(9, 6)

84

Sekarang, kita dapat memverifikasi hasil perhitungan sebelumnya. (Karena `P` adalah rasio sedangkan `choose` merupakan jumlah asli,
maka nilai `P` harus dikalikan dengan `N`, besar dari ruang sampel, agar kedua sisi berada pada skala yang sama.)

> *Semua bola merah*.

In [11]:
N = len(U6)

N * P(select('R', 6), U6) == choose(9, 6)

True

> *3 biru, 1 merah, dan 2 putih*.

In [12]:
N * P(select('B', 3) & select('W', 2) & select('R', 1), U6) == choose(6, 3) * choose(8, 2) * choose(9, 1)

True

In [13]:
N * P(select('W', 4), U6) == choose(8, 4) * choose(6 + 9, 2)  # (6 + 9 non-white balls)

True